# Make catalogue of sources above limit

We want a definitive list of bright objects. Here we look for all pixels above a level and make a catalogue of 'sources' of contiguous regions of such pixels.

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table,vstack
from astropy.wcs import WCS
from astropy.io import fits
from scipy.ndimage import label, generate_binary_structure



In [ ]:
maps = glob.glob('./data/*SPIRE500_v1.0.fits')

In [ ]:
maps

In [ ]:
!mkdir figs

In [ ]:
def makeCat(mapName,limit=100.e-3):
    "open map and make catalogue of sources"
    #mapName=maps[2]
    #limit= 100.e-3

    f=fits.open(mapName)
    w = WCS(f[1].header)
    #open the image as numpy array
    image = f[1].data
    #make a mask above the limit
    aboveLim = image>limit
    #show the whole image

    #get contiguous regions
    labeled_array, num_features = label(aboveLim)
    
    #make the catalogue of sources
    cat = Table()
    cat['id']=np.arange(1,num_features)
    cat['field']=mapName.split('/')[-1].split('_')[0]
    cat['field'] = cat['field'].astype('U50')
    cat['num_pixels']=[np.sum(labeled_array==i) for i in cat['id']]
    # cat['pixel_values']
    cat['pixel_x'] = [np.where(labeled_array == i)[0] for i in cat['id']]
    cat['pixel_y'] = [np.where(labeled_array == i)[1] for i in cat['id']]
    cat['pixel_ra'] =[
        w.pixel_to_world(
            np.where(labeled_array == i)[0],np.where(labeled_array == i)[1]
        ).ra.deg 
        for i in cat['id']]
    cat['pixel_dec'] =[
        w.pixel_to_world(
            np.where(labeled_array == i)[0],np.where(labeled_array == i)[1]
        ).dec.deg 
        for i in cat['id']]
    cat['pixel_value']= [image[row['pixel_x'],row['pixel_y']] for row in cat]
    cat['peak_pixel_value'] = [np.max(row['pixel_value']) for row in cat]
    cat['peak_index'] = [int(np.where(np.isclose(row['pixel_value'],row['peak_pixel_value']))[0]) for row in cat]
    cat['peak_pixel_x'] = [row['pixel_x'][row['peak_index']] for row in cat]
    cat['peak_pixel_y'] = [row['pixel_y'][row['peak_index']] for row in cat]
    cat['peak_pixel_ra'] = [row['pixel_ra'][row['peak_index']] for row in cat]
    cat['peak_pixel_dec'] = [row['pixel_dec'][row['peak_index']] for row in cat]
    
    print("""Pixels above {} Jy: {}. 
Number of sources: {}. 
Number of sources with more than one pixel:{}""".format(
        limit,np.sum(aboveLim.flatten()), 
        num_features,
        np.sum(cat['num_pixels']>1)
    ))
    plt.figure(figsize=(8,8),dpi=250)
    plt.imshow(
        image, 
        origin='lower',
        clim=(
            np.nanpercentile(image.ravel(),0.1), 
            np.nanpercentile(image.ravel(),99.9)
            ))
    #plt.ylim([0,np.max(image.shape[1])])
    plt.scatter(cat['peak_pixel_y'],cat['peak_pixel_x'],s=200, facecolors='none', edgecolors='r')
    
    plt.savefig('./figs/{}.png'.format(mapName.split('/')[-1].replace('.fits',''))) 
    plt.show()
    cat[cat['num_pixels']>1].show_in_notebook()

    return cat
    
cat=makeCat(maps[2])

In [ ]:
!mkdir -p data/cats

In [ ]:
fullCat=Table()
for m in maps:
    for limit in [80.,100.]:
        cat=makeCat(m,limit =limit*1.e-3)
        cat.write(
            m.replace('data','data/cats').replace('.fits','_{}_pix_cat.vot').format(limit),
            format='votable',overwrite=True)
        nonObjCols=[]
        for c in cat.colnames:
            if str(cat[c].dtype)!='object':
                nonObjCols.append(c)
        cat[nonObjCols].write(
            m.replace('data','data/cats').replace('.fits','_{}_pix_cat.fits').format(limit),
            overwrite=True)
        if limit>90:
            fullCat=vstack([fullCat,cat[nonObjCols]])

In [ ]:
np.sum(fullCat['num_pixels']>1),len(fullCat)

In [ ]:
fullCat[:5]

In [ ]:
fullCat.write('./figs/fullPixelCat_100mjy.fits',
            overwrite=True)